In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
import plotly.express as px
from numpy.linalg import eigvalsh

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior

In [40]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

n_roots = 350
gamma = 0.001

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    gamma=gamma,
    random_state=0,
)

# Calculate the prior covariance matrix
prior_covariance_matrix = smoothness_prior.prior_covariance()

# Display the covariance matrix
print("Prior Covariance Matrix:")
pd.DataFrame(prior_covariance_matrix)

Prior Covariance Matrix:


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.923332e-02,-7.733981e-02,6.307759e-02,-1.056119e-02,-4.925186e-03,3.860345e-03,-4.687844e-04,1.412190e-04,-5.652407e-06,-8.951317e-09,...,-8.903904e-07,-2.226838e-07,-7.460313e-08,-3.060128e-08,-1.455381e-08,-4.032140e-09,-1.551998e-09,-5.414722e-10,0.0,0.0
1,-7.733981e-02,5.679626e-01,-4.655098e-01,5.334809e-02,5.266470e-03,-2.717466e-02,2.686339e-03,-1.166406e-03,2.433915e-06,5.867062e-08,...,-2.230224e-07,2.739579e-07,4.067845e-08,1.400886e-07,2.342933e-08,4.122129e-09,7.159206e-09,-1.077676e-09,0.0,0.0
2,6.307759e-02,-4.655098e-01,4.361016e-01,-6.360372e-02,-3.152101e-02,-3.265077e-03,-2.990228e-03,-9.790614e-05,5.355017e-06,-4.759650e-08,...,-7.481403e-08,4.079242e-08,3.481549e-07,6.340573e-09,4.238315e-09,1.991011e-08,-6.384536e-09,-8.071372e-10,0.0,0.0
3,-1.056119e-02,5.334809e-02,-6.360372e-02,1.822517e-01,4.031082e-03,-3.473192e-03,-1.600420e-02,-1.117607e-04,4.676986e-06,-3.086493e-07,...,-3.089561e-08,1.401252e-07,6.475244e-09,-7.309325e-07,6.938482e-09,-4.816361e-09,-3.821763e-08,6.742120e-09,0.0,0.0
4,-4.925186e-03,5.266470e-03,-3.152101e-02,4.031082e-03,4.912379e-01,-5.636700e-03,-6.011122e-04,2.961776e-03,-2.225908e-05,-3.078057e-08,...,-1.467463e-08,2.317646e-08,6.329792e-09,1.161525e-08,-6.266658e-08,-2.877925e-08,-1.192663e-08,3.934778e-08,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-4.032140e-09,4.122129e-09,1.991011e-08,-4.816361e-09,-2.877925e-08,-5.367563e-09,1.188485e-07,8.081673e-09,4.684129e-10,2.175821e-12,...,3.329373e-09,7.263300e-10,3.190487e-10,9.974986e-11,4.717051e-11,5.000000e-01,4.442039e-12,-4.208268e-13,0.0,0.0
96,-1.551998e-09,7.159206e-09,-6.384536e-09,-3.821763e-08,-1.192663e-08,1.143802e-07,5.113476e-08,5.366549e-09,2.062033e-10,1.193519e-12,...,1.335143e-09,2.486973e-10,8.097212e-11,4.285707e-11,1.592026e-11,4.442039e-12,5.000000e-01,1.137978e-13,0.0,0.0
97,-5.414722e-10,-1.077676e-09,-8.071372e-10,6.742120e-09,3.934778e-08,-1.483923e-09,-1.105712e-10,4.448742e-10,1.299538e-11,2.932939e-14,...,6.831907e-11,1.784975e-11,-6.675969e-12,1.439963e-12,1.149526e-12,-4.208268e-13,1.137978e-13,5.000000e-01,0.0,0.0
98,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.5,0.0


In [41]:
np.linalg.eigvalsh(prior_covariance_matrix)

array([4.72619737e-05, 3.04892892e-04, 8.77472742e-04, 5.90118509e-03,
       1.23581793e-02, 3.57852057e-02, 7.35709704e-02, 9.08717357e-02,
       1.71025054e-01, 2.46738107e-01, 2.63433709e-01, 3.81005340e-01,
       3.81193300e-01, 4.79989122e-01, 4.88880655e-01, 4.89418959e-01,
       4.95970683e-01, 4.96791155e-01, 4.98571773e-01, 4.98881619e-01,
       4.99143272e-01, 4.99718980e-01, 4.99796117e-01, 4.99840205e-01,
       4.99867073e-01, 4.99910010e-01, 4.99912000e-01, 4.99922253e-01,
       4.99953860e-01, 4.99988656e-01, 4.99993763e-01, 4.99996789e-01,
       4.99997954e-01, 4.99998707e-01, 4.99999151e-01, 4.99999327e-01,
       4.99999613e-01, 4.99999678e-01, 4.99999752e-01, 4.99999779e-01,
       4.99999784e-01, 4.99999830e-01, 4.99999865e-01, 4.99999901e-01,
       4.99999936e-01, 4.99999953e-01, 4.99999968e-01, 4.99999981e-01,
       4.99999997e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
      

In [42]:
# Create a heatmap for the prior covariance matrix
fig1 = px.imshow(
    prior_covariance_matrix,
    labels=dict(x="Index", y="Index", color="Cov."),
    title="Smoothness Prior Covariance Matrix",
    color_continuous_scale="Viridis"
)

# Update layout for better visualization
fig1.update_layout(
    width=800,
    height=800,
    title_x=0.5,
    xaxis_title="Maturity and Strike Indices",
    yaxis_title="Maturity and Strike Indices",
)

# Show the figure
fig1.show()

In [43]:
# Compute the eigenvalues of the prior covariance matrix
eigenvalues = np.linalg.eigvalsh(prior_covariance_matrix)

# Sort eigenvalues by their absolute values in descending order
sorted_eigenvalues = np.sort(eigenvalues)[::-1]

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(sorted_eigenvalues) / np.sum(sorted_eigenvalues)

# Create a bar plot for the sorted eigenvalues (original values) and a line plot for cumulative explained variance
fig2 = go.Figure()

# Bar plot for eigenvalues
fig2.add_trace(
    go.Bar(
        x=np.arange(1, len(sorted_eigenvalues) + 1),
        y=np.abs(sorted_eigenvalues),
        name="Eigenvalues",
    )
)

# Line plot for cumulative explained variance
fig2.add_trace(
    go.Scatter(
        x=np.arange(1, len(cumulative_explained_variance) + 1),
        y=cumulative_explained_variance,
        name="Cumulative Explained Variance",
        yaxis="y2",
        mode="lines",
    )
)

# Add figure layout
fig2.update_layout(
    title="Eigenvalues and Cumulative Explained Variance",
    xaxis=dict(title="Index of Eigenvalues"),
    yaxis=dict(title="Eigenvalues"),
    yaxis2=dict(
        title="Cumulative Explained Variance",
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    width=900,
    height=900,
    title_x=0.5,
    legend=dict(x=0.7, y=0.9),
)

# Show the figure
fig2.show()

In [44]:
eigenvalues

array([4.72619737e-05, 3.04892892e-04, 8.77472742e-04, 5.90118509e-03,
       1.23581793e-02, 3.57852057e-02, 7.35709704e-02, 9.08717357e-02,
       1.71025054e-01, 2.46738107e-01, 2.63433709e-01, 3.81005340e-01,
       3.81193300e-01, 4.79989122e-01, 4.88880655e-01, 4.89418959e-01,
       4.95970683e-01, 4.96791155e-01, 4.98571773e-01, 4.98881619e-01,
       4.99143272e-01, 4.99718980e-01, 4.99796117e-01, 4.99840205e-01,
       4.99867073e-01, 4.99910010e-01, 4.99912000e-01, 4.99922253e-01,
       4.99953860e-01, 4.99988656e-01, 4.99993763e-01, 4.99996789e-01,
       4.99997954e-01, 4.99998707e-01, 4.99999151e-01, 4.99999327e-01,
       4.99999613e-01, 4.99999678e-01, 4.99999752e-01, 4.99999779e-01,
       4.99999784e-01, 4.99999830e-01, 4.99999865e-01, 4.99999901e-01,
       4.99999936e-01, 4.99999953e-01, 4.99999968e-01, 4.99999981e-01,
       4.99999997e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
      

In [45]:
smoothness_prior.sample_smooth_surfaces(9)

array([[ 5.91968279e-04, -1.01677548e-01,  8.79748035e-02,
         5.61363065e-01, -1.74920968e-01,  1.31622980e-02,
         9.72541698e-02, -2.48927379e+00,  1.42368558e-01,
        -8.18732453e-01,  1.08080142e-01, -4.83389372e-01,
         1.76785716e-01, -1.49198064e-01, -9.19792379e-03,
        -2.15919664e-01,  4.65608293e-01, -5.72927588e-01,
        -4.74719416e-02,  4.38722901e-01, -2.74468695e-01,
         5.60062565e-01,  1.70554782e-01,  5.15247706e-03,
         1.91189639e-01, -2.92966729e-01,  8.77834068e-01,
        -3.28822349e-02,  1.06200759e+00, -8.86609317e-01,
         3.88284032e-01, -4.24755138e-01, -3.81184765e-01,
        -1.79508911e+00,  4.02860319e-01, -3.81912630e-01,
        -2.29106833e-01,  1.89598919e-01,  9.05044578e-01,
         2.68882584e-01, -2.98022457e-01,  1.13394880e-01,
         9.50764597e-01,  1.79778595e-01,  4.50928585e-01,
        -4.49284425e-01, -1.35804541e-01,  1.08966996e+00,
         1.16663229e-02, -3.60488980e-01,  1.27009947e-0